# EXTRACTION
### webscraps inec database

In [130]:
#driver for Chrome
from selenium.webdriver import Chrome
#selector for dropdown
from selenium.webdriver.support.ui import Select
#waits until a process or page is accessible 
from selenium.webdriver.support.ui import WebDriverWait
#By.XPATH, By.CSS_SELECTOR etc
from selenium.webdriver.common.by import By
#manipulates properties of chrome driver like add fake user agent
from selenium.webdriver.chrome.options import Options
#certain conditions need to be met before a process begins
from selenium.webdriver.support import expected_conditions as EC
#this changes the useragent name which reduces reCAPTCHA verification
from fake_useragent import UserAgent
#allows code to sleep for a little time until process is completely done
import time

options = Options()
ua = UserAgent()
userAgent = ua.random
options.add_argument(f'user-agent={userAgent}')

#access to the path of your chromedriver.exe
driver = Chrome('C:/Users/HP/Downloads/inec_scum/chromedriver.exe')
#the homepage since you have to sign in first before navigating to the required page for sign in
driver.get("https://cvr.inecnigeria.org/Home/start")
#waits until your page loads completely before sign in process begins
driver.implicitly_wait(20)

#sign in cuse using xpath and the id of the html properties as key as well ass value for login button
driver.find_element_by_xpath("""//*[@id="LoginEmail"]""").send_keys('use your own email')
driver.find_element_by_xpath("""//*[@id="LoginPassword"]""").send_keys('use your password')
driver.find_element_by_xpath("""//*[@value="Log in"]""").click()

#navigates to the page for inputting state, LGA,AREA id and polling unit
driver.get("https://cvr.inecnigeria.org/VotersRegister")
driver.implicitly_wait(6)

select1 = Select(driver.find_element_by_id('VoterRegisterStateId'))
#selecting by value instead of state name
select1.select_by_value('1')
select2 = Select(driver.find_element_by_id('VoterRegisterLocalGovernmentId'))
select2.select_by_value('1')
select3 = Select(driver.find_element_by_id('VoterRegisterRegistrationAreaId'))
#at a point the values sudden increase exponentially like from 22 to 61373
select3.select_by_value('1')
select4 = Select(driver.find_element_by_id('VoterRegisterPollingUnitId'))
select4.select_by_value('1')

#waits for page is ready then switches to the reCAPTCHA iframe
WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH,"//iframe[@title='reCAPTCHA']")))
#clicks on the reCAPTCHA button and it either verifies without image test or it doesn't. currently at 8 out of 10 times with 8 being verification without image test
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.recaptcha-checkbox-border"))).click()


time.sleep(5)
#switches from reCAPTCHA iframe back to default reCAPTCHA else your code will gives error about not recognizing the next xpath
driver.switch_to.default_content()
#clicks on display button
driver.find_element_by_xpath("""//*[@value="DISPLAY REGISTER"]""").click()
driver.implicitly_wait(20)
names = []
loca = []
#required data can be found in the table with class name below that contains the name, gender, date of birth and vin for everyone in that page
contents =driver.find_elements_by_class_name("table.table-condensed.table-borderless.condensed-rows.voter")
for i in contents:
    name = i.get_attribute('innerText')
    names.append(name)
location =driver.find_elements_by_class_name("table.table-bordered.table-striped.afterpagebreak.bg-white")
for s in location:
    local= s.get_attribute('innerText')
    loca.append(local)
    

#ends the driver process
driver.quit()

['LGA:\nABA NORTH\tPOLLING UNIT:\nRAILWAY QUARTERS - RAILWAY QUARTERS I\nWARD:\nEZIAMA\tDELIM:\n01-01-01-001\tRECORDS:\n968']


# TRANSFORM

In [144]:
import pandas as pd
#turns the area list into a pandas dataframe and changes '\n' and '\t' to ','
#also changes the other repeated headers to nothing
af = pd.DataFrame(loca,columns =['area'])
af.area = af.area.apply(lambda x: x.replace('\n', ','))
af.area = af.area.apply(lambda x: x.replace('\t', ','))
af.area = af.area.apply(lambda x: x.replace('LGA:,', ''))
af.area = af.area.apply(lambda x: x.replace(',POLLING UNIT:', ''))
af.area = af.area.apply(lambda x: x.replace(',WARD:', ''))
af.area = af.area.apply(lambda x: x.replace(',DELIM:', ''))
af.area = af.area.apply(lambda x: x.replace(',RECORDS:', ''))

af = af['area'].str.split(",", n = 5, expand = True)
af.columns = ['LGA', 'Polling_unit', 'Ward', 'Delim','Records']
af

,LGA,Polling_unit,Ward,Delim,Records
0,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968


In [145]:
import pandas as pd

df = pd.DataFrame(names,columns =['Names'])
df = df.replace('\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0', '')
df.Names = df.Names.apply(lambda x: x.replace('\t\n\t', ','))
df.Names = df.Names.apply(lambda x: x.replace('\n\n', ','))
df.Names = df.Names.apply(lambda x: x.replace('VIN:', ''))
df.Names = df.Names.apply(lambda x: x.replace('DOB-Y:', ''))
df.Names = df.Names.apply(lambda x: x.replace('GENDER:', ','))
df = df['Names'].str.split(",", n = 7, expand = True)
df.columns = ['no_', 'First_name', 'Last_name', 'VIN','DOB','Gender']
df

,no_,First_name,Last_name,VIN,DOB,Gender
0,1,ABA,ANTHONY,90F5B1FFD8295703336,1967,M
1,2,ABANGWU,BLESSING,90F5B1FFD8295354202,1958,F
2,3,Abdu,Ado,FA5B3EC1DB523372976,1968,M
3,4,Abdulkadir,Mohadmmed D.,90F5B124EF523523270,1983,M
4,5,ABDULLAHI,ABDULSAMAD A.,INC22000000014918081,1998,M
5,6,Abdullahi,Ibrahim I.,90F5B0868B522746607,1996,M
6,7,ABDULLAHI,JAMILU,90F5B05E53501493322,1994,M
7,8,Abdullahi,Mugammad B.,FA5B3EC240523551231,1986,M
8,9,ABDULLATEEF,QASIM O.,90F5B1FCBF504596498,1986,M
9,10,ABDULRAMAN,FATIMOH D.,90F5B1FCBF506416337,1988,F


In [152]:
import pandas as pd
#concats both df and af datafrmaes together
uf = pd.concat([df, af], axis=1)
#since the af dataframe only has one row that corresponds to the df counterpart, fill the nan values with anything above it
uf= uf.fillna(method='ffill')
uf

,no_,First_name,Last_name,VIN,DOB,Gender,LGA,Polling_unit,Ward,Delim,Records
0,1,ABA,ANTHONY,90F5B1FFD8295703336,1967,M,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968
1,2,ABANGWU,BLESSING,90F5B1FFD8295354202,1958,F,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968
2,3,Abdu,Ado,FA5B3EC1DB523372976,1968,M,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968
3,4,Abdulkadir,Mohadmmed D.,90F5B124EF523523270,1983,M,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968
4,5,ABDULLAHI,ABDULSAMAD A.,INC22000000014918081,1998,M,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968
5,6,Abdullahi,Ibrahim I.,90F5B0868B522746607,1996,M,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968
6,7,ABDULLAHI,JAMILU,90F5B05E53501493322,1994,M,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968
7,8,Abdullahi,Mugammad B.,FA5B3EC240523551231,1986,M,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968
8,9,ABDULLATEEF,QASIM O.,90F5B1FCBF504596498,1986,M,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968
9,10,ABDULRAMAN,FATIMOH D.,90F5B1FCBF506416337,1988,F,ABA NORTH,RAILWAY QUARTERS - RAILWAY QUARTERS I,EZIAMA,01-01-01-001,968


# LOADING